In [2]:
!pip install datasets transformer_lens wandb plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 72.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ActivationStoreParallel import ActivationsStore
from sparse_transcoder import SparseTranscoder
from transcoder_training_parallel import train_transcoder_on_language_model_parallel
from transcoder_runner_parallel import language_model_transcoder_runner_parallel
from dataclasses import dataclass
import transformer_lens
import torch
import wandb
from typing import Optional

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: biggs (biggs-University College London (UCL)). Use `wandb login --relogin` to force relogin


True

In [4]:
@dataclass
class UnifiedConfig():
    # Common settings
    model_name: str = "gpt2-small"
    hook_point: str = "blocks.10.hook_resid_pre"
    ln: str = 'blocks.10.ln1.hook_scale'
    hook_point_layer: int = 10
    layer: int = 10
    d_in: int = 768
    d_out: int = 768
    n_head: int = 12
    d_head: int = 64
    dataset_path: str = "Skylion007/openwebtext"
    is_dataset_tokenized: bool = False
    training: bool = True
    attn_scores_normed = True
    
    # SAE Parameters
    expansion_factor: int = 12   # TODO: NOT being used??
    d_hidden: int = 2400
    b_dec_init_method: str = "mean"
    
    # Training Parameters
    lr: float = 1e-5
    reg_coefficient: float = 4e-6
    lr_scheduler_name: Optional[str] = None
    train_batch_size: int = 2048
    context_size: int = 256
    lr_warm_up_steps: int = 5000
    
    # Activation Store Parameters
    n_batches_in_buffer: int = 128
    total_training_tokens: int = 20_000 * 5_000
    store_batch_size: int = 32
    use_cached_activations: bool = False
    
    # Resampling protocol
    feature_sampling_method: str = 'none'
    feature_sampling_window: int = 1000
    feature_reinit_scale: float = 0.2
    resample_batches: int = 1028
    dead_feature_window: int = 50000
    dead_feature_threshold: float = 1e-6
    
    # WANDB
    log_to_wandb: bool = True
    log_final_model_to_wandb: bool = False
    wandb_project: str = "sparsification"
    wandb_entity: Optional[str] = None
    wandb_log_frequency: int = 50
    entity: str = "biggs-University College London (UCL)"
    
    # Misc
    device: str = "cuda"
    eps: float = 1e-7
    seed: int = 42
    reshape_from_heads: bool = True
    n_checkpoints: int = 10
    checkpoint_path: str = "checkpoints"
    dtype: torch.dtype = torch.float32
    run_name: str = "qk_parallel"
    
    # Query-specific settings
    hook_transcoder_in_q: str = "blocks.10.hook_resid_pre"
    hook_transcoder_out_q: str = "blocks.10.attn.hook_q"
    target_q: str = "blocks.10.attn.hook_q"
    type_q: str = "resid_to_queries"
    
    # Key-specific settings
    hook_transcoder_in_k: str = "blocks.10.hook_resid_pre"
    hook_transcoder_out_k: str = "blocks.10.attn.hook_k"
    target_k: str = "blocks.10.attn.hook_k"
    type_k: str = "resid_to_keys"

cfg = UnifiedConfig()
cfg.run_name = f"{cfg.d_hidden}_{cfg.reg_coefficient}_{cfg.lr}"

In [ ]:
sparse_transcoder_Q, sparse_transcoder_K = language_model_transcoder_runner_parallel(cfg)

Loaded pretrained model gpt2-small into HookedTransformer
Dataset is not tokenized! Updating config.


Setting up optimizer.
Initializing autoencoders.
Reinitializing b_dec with mean of activations
Previous distances: 27.712806701660156
New distances: 19.674325942993164
Reinitializing b_dec_out with mean of activations
Previous distances: 36.17184066772461
New distances: 28.02600860595703
Reinitializing b_dec with mean of activations
Previous distances: 27.712806701660156
New distances: 19.674325942993164
Reinitializing b_dec_out with mean of activations
Previous distances: 41.437225341796875
New distances: 31.178068161010742


Training Transcoders:  10%|█         | 10041344/100000000 [03:00<59:55, 25021.97it/s, Loss=1.876]  

Saved model to checkpoints/10000384_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/10000384_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt


Training Transcoders:  20%|██        | 20037632/100000000 [05:59<31:51, 41833.27it/s, Loss=1.381]  

...

Training Transcoders:  50%|█████     | 50038784/100000000 [14:58<06:03, 137317.20it/s, Loss=1.260] 

Saved model to checkpoints/50001920_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/50001920_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt


Training Transcoders:  60%|██████    | 60039168/100000000 [17:59<04:16, 156028.72it/s, Loss=1.228] 

Saved model to checkpoints/60000256_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/60000256_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt


Training Transcoders:  67%|██████▋   | 67108864/100000000 [20:04<02:45, 198168.13it/s, Loss=1.223] 

In [13]:
sparse_transcoder_K.get_name()

'sparse_transcoder_gpt2-small_resid_to_keys_2400'

In [5]:
q = SparseTranscoder.load_from_pretrained("checkpoints/final_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt", True)

In [6]:
q.W_enc

Parameter containing:
tensor([[-0.0274, -0.0142,  0.0207,  ...,  0.0160,  0.0216, -0.0334],
        [ 0.0333, -0.0227, -0.0014,  ...,  0.0028, -0.0333, -0.0182],
        [ 0.0353, -0.0476, -0.0202,  ...,  0.0003, -0.0036, -0.0372],
        ...,
        [-0.0498,  0.0323, -0.0045,  ..., -0.0020,  0.0064, -0.0032],
        [ 0.0111, -0.0008,  0.0419,  ...,  0.0256,  0.0485,  0.0258],
        [ 0.0077, -0.0356,  0.0428,  ...,  0.0197,  0.0268,  0.0358]],
       device='cuda:0', requires_grad=True)